# Projeto 1 - Ciência dos Dados

Nome: Carlos Simodo

Nome: Luiz Fernando da Silva Borges

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção dos tweets

## Preparando o ambiente no jupyter:

In [12]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [13]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @hoxie_reader ]***

In [14]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [32]:
#Produto escolhido:
produto = 'Lula'

#Quantidade mínima de mensagens capturadas:
n = 3000
#Quantidade mínima de mensagens para a base de treinamento:
t = 668

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [33]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break


In [42]:
#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

print(len(msgs))

1002


In [43]:
msgs = list(set(list(msgs)))

len(msgs)

1002

Uma vez que a API capturou 2000 tweets, decidimos prosseguir com 1002 tweets no total. 


In [44]:
lista_cortada = msgs[:1002]
msgs = lista_cortada

len(msgs)

1002

In [53]:
#Verifica se o arquivso não existem para não substituir um conjunto pronto
if not os.path.isfile('./mensagens_1.xlsx') and os.path.isfile('./mensagens_2.xlsx'):
    
    #pega a quantidade de metade das mensagens escolhidas (deve ser par)
    metade_mensagens = int(len(msgs)/2)
    
    #Abre o arquivo para escrita
    writer1 = pd.ExcelWriter("mensagens_1A.xlsx")
    writer2 = pd.ExcelWriter("mensagens_2A.xlsx")
    
    #divide o conjunto de mensagens totais em duas planilhas, pois trata-se de um projeto em dupla
    dft1 = pd.DataFrame({"metade_1": pd.Series(msgs[:metade_mensagens])})
    dft2 = pd.DataFrame({"metade_2": pd.Series(msgs[metade_mensagens:])})
    
    #converte os data frames para arquivos .xlsx
    dft1.to_excel(excel_writer = writer1, index = False)
    writer1.save()
    
    dft2.to_excel(excel_writer = writer2, index = False)
    writer2.save()
    

In [94]:
df_mensagens_etiquetadas_1 = pd.read_excel('tabela_etiquetada_1.xlsx', index_col=0) 
df_mensagens_etiquetadas_2 = pd.read_excel('tabela_etiquetada_2.xlsx', index_col=0) 
df_tabela_etiquetada = pd.concat([df_mensagens_etiquetadas_1, df_mensagens_etiquetadas_2])

Salvando os dados em uma planilha Excel:

In [95]:
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))
    
    df_treinamento = df_tabela_etiquetada[:t]
    df_treinamento.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = True) 
    
    df_teste = df_tabela_etiquetada[t:]
    df_teste.to_excel(excel_writer = writer, sheet_name = 'Teste', index = True) 
    
    writer.save()
    

In [ ]:
##########################################################################################

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**